In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df=pd.read_csv('/kaggle/input/airpressure/Folds5x2_pp.csv')
df.head()

Features consist of hourly average ambient variables
- Temperature (AT) in the range 1.81°C and 37.11°C,
- Ambient Pressure (AP) in the range 992.89-1033.30 milibar,
- Relative Humidity (RH) in the range 25.56% to 100.16%
- Exhaust Vacuum (V) in teh range 25.36-81.56 cm Hg
- Net hourly electrical energy output (EP) 420.26-495.76 MW
The averages are taken from various sensors located around the plant that record the ambient variables every second. The variables are given without normalization. 

In [ ]:
df.shape

The dataset contains 9568 data points collected from a Combined Cycle Power Plant over 6 years (2006-2011), when the power plant was set to work with full load. Features consist of hourly average ambient variables Temperature (T), Ambient Pressure (AP), Relative Humidity (RH) and Exhaust Vacuum (V) to predict the net hourly electrical energy output (EP) of the plant.
A combined cycle power plant (CCPP) is composed of gas turbines (GT), steam turbines (ST) and heat recovery steam generators. In a CCPP, the electricity is generated by gas and steam turbines, which are combined in one cycle, and is transferred from one turbine to another. While the Vacuum is colected from and has effect on the Steam Turbine, the other three of the ambient variables effect the GT performance.

In [ ]:
# Importing Libraries. 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import linear_model
import seaborn as sns

In [ ]:
#summary statistics:
df.info()


* All the independent variables are numerical data type and it doesnt have any null values.
* The dependent variable is PE - Electrical energy ouput is also an numeric type without null values.
* Since, the value is numerical and contineous. So, Linear regression model is used to predict the value of the power.

In [ ]:
df.describe()

* According to the statistical description, all the independent variables doesnt have any kind of outliers.
* This is identified by the mean value is close towards the 50% of the IQR value.

In [ ]:
fig, axes = plt.subplots(2,2)
for i,el in enumerate(list(df.columns.values)[:-1]):
    a = df.boxplot(el, ax=axes.flatten()[i])


* From this we can say that AP and RH has some outliers.
* We need to analyze that wheather it is outlier or extreme values.

In [ ]:
#checking for the distribution using violin plot:
for i in range(len(df.columns)):
    sns.distplot(df.iloc[:,i])
    plt.show()

* AT - Temperature is more or less normally distributed.
* AP - Pressure is normally distrubuted
* RH - Humidity is left skewed. 
* PE - It has 2 peaks with normally distributed plot.


In [ ]:
#checking linearity:
sns.pairplot(df)
print(df.corr(method="spearman"))

col=df.columns
print(col)

* Relation between AT vs V has strong positive correlation as 0.8 - says that temperature and vaccum has strong positive correlation.
* AT vs PE has strong negative correlation as -0.9 - says that temeperature vs electrical output as strong negative correlation. In which, ambient temperature increases the energy output is decreses. 
* V vs AT has strong positive correlation as 0.8
* V vs PE has strong negative correlation as -0.8 - In which, the exhaust vaccum is high then the energy output is very low and the vice versa

In [ ]:
#normalizing the data:
df_nor=preprocessing.normalize(df)
print(df_nor)
df_nor=pd.DataFrame(df_nor)
print(df_nor)
sns.pairplot(df_nor)
df_nor.corr()
df_nor.columns = df.columns
df_nor.head()
x=df_nor.iloc[:,0:4]
y=df_nor.iloc[:,4]

* After normalizing the data,
    * The correlation of AP is showing a strong negative correlation.

In [ ]:
#splitting train set:
X_train,X_test,y_train,y_test=model_selection.train_test_split(x,y,test_size=0.2)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
#fitting linear model:
lm=linear_model.LinearRegression()
model=lm.fit(X_train,y_train)
pred=lm.predict(X_train)
print(pred)
print(model.coef_)
print(model.intercept_)

In [ ]:
#checking accuracy:
from sklearn.metrics import r2_score
print(r2_score(pred,y_train))
predd = lm.predict(X_test)
print(r2_score(predd,y_test))

In [ ]:
sns.regplot(x=predd, y=y_test, lowess=True, line_kws={'color': 'red'})
plt.title('Observed vs. Predicted Values', fontsize=16)
